<a href="https://colab.research.google.com/github/TalissaMoura/AmazonC4DatasetAugmented/blob/main/notebooks/2_pipeline_for_calculate_the_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2 - Pipeline for measure the metrics for the hard queries generated

In [1]:
!pip install outlines

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.4 MB/s eta 0:00:00


In [22]:
import os, json, re, torch
import outlines
from huggingface_hub import login,snapshot_download, upload_file, HfApi
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
from google.colab import userdata
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq
from pydantic import BaseModel,ValidationError
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [3]:
from google.colab import userdata
from datasets import load_dataset
import pathlib
hf_token = userdata.get('HF_TOKEN')
df_amazonc4 = load_dataset('McAuley-Lab/Amazon-C4')['test'].data.table
snapshot_download(
    repo_id='Talissa/AmazonC4Augmented',
    repo_type='dataset',
    local_dir='datasets',
    allow_patterns=['datasets/procesed/hard_negative_queries/all_Beauty_part_0.parquet'],
    token=hf_token,
)
# setup hugging faces
hf_api = HfApi(token=hf_token)

README.md: 0.00B [00:00, ?B/s]

test.csv:   0%|          | 0.00/12.5M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/21223 [00:00<?, ? examples/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

datasets/procesed/hard_negative_queries/(…):   0%|          | 0.00/5.71k [00:00<?, ?B/s]

# 2 - Functions

In [53]:
path_procesed_hnq = pathlib.Path('/content/datasets/contraint_lists_for_generated_queries')

if not pathlib.Path(path_procesed_hnq).exists():
  pathlib.Path.mkdir(path_procesed_hnq,exist_ok=True,parents=True)

class Constraints(BaseModel):
    constraints: list[str]
class Intents(BaseModel):
    intents: list[str]


def extract_json_from_markdown(response_text: str) -> str:
    """Extract JSON from markdown code blocks"""
    # Look for ```json ... ``` or ``` ... ```
    pattern = r'```(?:json)?\s*(.*?)\s*```'
    matches = re.findall(pattern, response_text, re.DOTALL)

    if matches:
        # Return the first JSON block found
        return matches[0].strip()
    else:
        # If no markdown blocks, return original text
        return response_text

def clean_and_parse_json(response_text: str,pydantic_model:BaseModel) -> BaseModel | None:
    # Step 1: Extract JSON from markdown if present
    json_content = extract_json_from_markdown(response_text)

    # Step 2: Remove any remaining backticks or markdown artifacts
    json_content = json_content.strip()

    # Step 3: Handle potential escaping issues
    if json_content.startswith('"') and json_content.endswith('"'):
        # Unescape if it's still a string representation
        try:
            json_content = json.loads(json_content)
        except json.JSONDecodeError:
            pass  # Keep as is if it's not double-escaped

    # Step 4: Parse and validate
    try:
        if isinstance(json_content, str):
            return pydantic_model.model_validate_json(json_content)
        else:
            return pydantic_model.model_validate(json_content)
    except ValidationError as e:
        print(f"Validation error: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        return None

from pydantic import ValidationError
import time

def generate_and_parse_with_retry(
    model,
    tokenizer,
    messages,
    pydantic_model,
    max_attempts=3,
    delay=0.5  # optional: to avoid overwhelming GPU or API rate limits
):
    for attempt in range(1, max_attempts + 1):
        # Tokenize and generate
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=2048
        )
        # Trim input part
        generated_ids = [
            output_ids[len(input_ids):]
            for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        print(response)

        # Try parsing
        try:
            parsed = clean_and_parse_json(response, pydantic_model)
            print(parsed)
            if parsed is not None:
                return parsed  # success!
        except ValidationError:
            pass  # will retry

        # Optional: small delay before next attempt
        if attempt < max_attempts:
            time.sleep(delay)

    # All attempts failed
    return None  # or raise an exception, depending on desired behavior


def generate_constraint_list(df_queries,
                             model,
                             tokenizer,
                             pydantic_model=BaseModel,
                             system_template_path=None,
                             user_template_path=None,
                             save_local=True):
  user_template = outlines.Template.from_file(f"{user_template_path}")
  system_template = outlines.Template.from_file(f"{system_template_path}")
  batches = df_queries.to_batches()
  for i, batch in enumerate(batches):
      new_rows = []
      for row in batch.to_pylist():
          query = row['hard_negative_query']

          messages = [
              {"role": "system", "content": system_template()},
              {"role": "user", "content": user_template(user_query=query)}
          ]

          # Try up to 3 times to get a valid parse
          response_treated = generate_and_parse_with_retry(
              model=model,
              tokenizer=tokenizer,
              messages=messages,
              pydantic_model=pydantic_model,
              max_attempts=3
          )

          if response_treated:
            new_rows.append(response_treated)
          else:
            new_rows.append([""])


  batch_pylist = batch.to_pylist()
  new_data = []
  for new_row,data in zip(new_rows,batch_pylist):
      data["constraints"] = new_row.__dict__["constraints"]
      new_data.append(data)

  if save_local:
    batch = pa.Table.from_pylist(new_data)
    pq.write_table(batch, f"/content/datasets/contraint_lists_for_generated_queries/all_Beauty_part_{i}.parquet")
    print(f"Batch {i} done.")


def generate_intent_list(df_queries,
                         model,
                         tokenizer,
                         pydantic_model=BaseModel,
                         text_type="review",
                         system_template_path=None,
                         user_template_path=None,
                         save_local=True):

  user_template = outlines.Template.from_file(f"{user_template_path}")
  system_template = outlines.Template.from_file(f"{system_template_path}")
  batches = df_queries.to_batches()
  for i, batch in enumerate(batches):
      new_rows = []
      for row in batch.to_pylist():
          if text_type == "review":
            text = row['ori_semi_positive_review']
          else:
            text = row['hard_negative_query']
          messages = [
              {"role": "system", "content": system_template()},
              {"role": "user", "content": user_template(text=text,text_type=text_type)}
          ]

          response_treated = generate_and_parse_with_retry(
              model=model,
              tokenizer=tokenizer,
              messages=messages,
              pydantic_model=pydantic_model,
              max_attempts=3
          )

          if response_treated:
              new_rows.append({
                  "text_type":"review",
                   "intents":response_treated.__dict__["intents"]
              })
          else:
              new_rows.append(
                  {
                  "text_type":"search_query",
                   "intents":response_treated.__dict__["intents"]
              })



  batch_pylist = batch.to_pylist()
  new_data = []
  for new_row,data in zip(new_rows,batch_pylist):
    if text_type == "review":
      data["user_review_intent"] = new_row["intents"]
    else:
      data["user_query_intent"] = new_row["intents"]
    new_data.append(data)

  if save_local:
    batch = pa.Table.from_pylist(new_data)
    pq.write_table(batch, f"/content/datasets/user_intent/{text_type}/all_Beauty_part_{i}.parquet")
    print(f"Batch {i} done.")


def load_model(model_name):
  tokenizer = AutoTokenizer.from_pretrained(model_name,fix_mistral_regex=True,device_map="cpu",)
  model = AutoModelForCausalLM.from_pretrained(model_name, dtype=torch.float16,device_map="auto")

  # 🔍 Validate placement
  if not next(model.parameters()).is_cuda:
      raise RuntimeError("Model failed to load on GPU!")
  print("✅ Model successfully on GPU.")
  return model,tokenizer


1 - Calculate similarity scores

In [5]:
model_eval = SentenceTransformer('all-MiniLM-L6-v2',device="cuda")

def add_queries_similarity_score(query,hard_negative_query):
  if not query or not hard_negative_query:
    return 0.0
  query_emb = model_eval.encode(query, convert_to_tensor=False)
  hard_negative_query_emb = model_eval.encode(hard_negative_query, convert_to_tensor=False)

  return cosine_similarity([query_emb], [hard_negative_query_emb])[0][0]


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2 - Calculate the Constraint Preservation Rate

In [7]:
df_reviews = pq.read_table('/content/datasets/datasets/procesed/hard_negative_queries/all_Beauty_part_0.parquet')

In [6]:
import outlines, pathlib

if not pathlib.Path('/content/prompts/').exists():
  pathlib.Path.mkdir('/content/prompts')

try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"/content/prompts/system_prompt_generate_query_restritions_list.txt"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    system_template = \
    """

    ```json
      {
        "task": "You are a constraint extractor. Given a product search query, identify and list *only* the explicit or strongly implied characteristics that the user wants to *exclude* or *avoid* in the product. Do not infer preferences unless clearly stated (e.g., 'no plastic', 'avoid heavy items'). Return an empty array if no such constraints exist.",
        "input_format": {
          "query": "string - the user's natural-language product search query",
          "text_type": "string - review or search query"
        },
        "output_type": "json",
        "output_format": {
          "constraints": [
            "string — a phrase that summarize what the user don't want in the product (i.e this product should not be small, this product should avoid the brand A, the user don't want overprice products .. )",
            "Avoid vague terms; prefer concrete, actionable constraints derived directly from the query."
          ]
        },
        "requirements": [
          "Constraints must be phrased as *what to avoid*, not what is desired.",
          "Normalize phrasing (e.g., use 'corded' instead of 'with a cord' if excluding cordless devices).",
          "Do NOT include positive requirements (e.g., 'durable' → ignore unless phrased negatively like 'not fragile').",
          "If ambiguity exists, prefer under-inclusion (i.e., only include unambiguous exclusions)."
        ]
      }
    ```

    """

    with open('/content/prompts/system_prompt_generate_query_restritions_list.txt', 'w+') as f:
          f.write(system_template)
          f.close()
          upload_file(
              path_or_fileobj=f"/content/prompts/system_prompt_generate_query_restritions_list.txt",
              path_in_repo=f"prompts/system_prompt_generate_query_restritions_list.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add system prompt for constraint preservation rate"
          )

No files have been modified since last commit. Skipping to prevent empty commit.


In [8]:
try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"prompts/user_prompt_generate_query_restritions_list"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    user_template = \
    """
    ```json
        {

          "query": {{user_query}}}

        }
    ```
    """

    with open('/content/prompts/user_prompt_generate_query_restritions_list.txt', 'w') as f:
          f.write(user_template)
          f.close()
          upload_file(
              path_or_fileobj=f"/content/prompts/user_prompt_generate_query_restritions_list.txt",
              path_in_repo=f"prompts/user_prompt_generate_query_restritions_list.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add user prompt for constraint preservation rate"
          )


No files have been modified since last commit. Skipping to prevent empty commit.


In [9]:
print('-- generate constraint list --')
MODEL_NAME = 'Qwen/Qwen2.5-3B-Instruct'
if "model" not in globals():
  model, tokenizer = load_model(MODEL_NAME)
else:
  print("Model already loaded")
generate_constraint_list(df_reviews,
                         model,tokenizer,
                         pydantic_model=Constraints,
                         system_template_path='/content/prompts/system_prompt_generate_query_restritions_list.txt',
                         user_template_path='/content/prompts/user_prompt_generate_query_restritions_list.txt',
                         )

-- generate constraint list --


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

✅ Model successfully on GPU.
```json
{
  "constraints": [
    "soft bristles"
  ]
}
```
constraints=['soft bristles']
```json
{
  "constraints": [
    "bristles aren't tough enough"
  ]
}
```
constraints=["bristles aren't tough enough"]
Batch 0 done.


In [ ]:
# adjust query to return the answer as json
# resave the files from the prompts for the first notebook

In [27]:
df_reviews_metrics = pq.read_table('/content/datasets/contraint_lists_for_generated_queries/all_Beauty_part_0.parquet')

def compute_consistent_score(x, y):
    # Handle edge cases
    if not x or not y:
        return 0.0  # or np.nan — up to you

    # Encode all sentences
    X = model_eval.encode(x, convert_to_tensor=False)
    Y = model_eval.encode(y, convert_to_tensor=False)

    # Compute cosine similarity matrix: shape = (len(X), len(Y))
    sims = cosine_similarity(X,Y)

    # For each constraint, get max similarity over negative embeddings
    max_sims = sims.max(axis=1)  # shape: (len(X),)

    # Return the biggest value over the similarities
    return float(np.max(max_sims))

def calculate_consistent_score(df_reviews):
  new_rows = []
  for batch in df_reviews.to_batches():
    for row in batch.to_pylist():
      negative_points = row['negative_points']
      constraint_list = row['constraints']
      row["consistent_score"] = compute_consistent_score(constraint_list,negative_points)
      row['consistent_hit'] = 1.0 if row['consistent_score'] >= 0.6 else 0.0
      new_rows.append(row)
  new_batch = pa.Table.from_pylist(new_rows)
  return new_batch

df_reviews_w_metrics = calculate_consistent_score(df_reviews_metrics)

In [28]:
print(df_reviews_w_metrics)

pyarrow.Table
item_id: string
ori_positive_review: string
ori_semi_positive_review: string
positive_points: list<item: string>
  child 0, item: string
negative_points: list<item: string>
  child 0, item: string
hard_negative_query: string
constraints: list<item: string>
  child 0, item: string
consistent_score: double
consistent_hit: double
----
item_id: [["B08MF3W9DZ","B08MF3W9DZ"]]
ori_positive_review: [["Great brushes  for curly hair Love these brushes for my girls who have super curly hair that’s very hard to untangle. Works great on wet or  dry Hair","Great brushes  for curly hair Love these brushes for my girls who have super curly hair that’s very hard to untangle. Works great on wet or  dry Hair"]]
ori_semi_positive_review: [["It’s a great brush, but it needs to be a tad bit tougher…. It combs, but when you need to get a tangle out, the bristles aren’t tough enough. I have to literally go get a different brush. This brush is very gentle. My daughter doesn’t complain…. But I nee

In [73]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# Load model ONCE (outside loops!)
# model = SentenceTransformer('all-MiniLM-L6-v2',device="cuda")  # Fast, good for this task

# def compute_consistent_score(negative_points, constraint_list):
#     # Handle edge cases
#     if not negative_points or not constraint_list:
#         return 0.0  # or np.nan — up to you

#     # Encode all sentences
#     neg_embs = model.encode(negative_points, convert_to_tensor=False)
#     const_embs = model.encode(constraint_list, convert_to_tensor=False)

#     # Compute cosine similarity matrix: shape = (len(neg), len(const))
#     sims = cosine_similarity(neg_embs, const_embs)

#     # For each negative point, get max similarity over constraints
#     max_sims = sims.max(axis=1)  # shape: (len(neg),)

#     # Return average of max similarities
#     return float(np.mean(max_sims))

eval_model = SentenceTransformer('all-MiniLM-L6-v2',device="cuda")

new_rows = []
for batch in df_reviews_metrics.to_batches():
  for row in batch.to_pylist():
    negative_points = row['negative_points']
    constraint_list = row['constraint_list']
    row["consistent_score"] =
    # row['consistent_hit'] = 0
    # for constraint in constraint_list:
    #   if constraint in negative_points:
    #     row['consistent_hit'] += 1
    # new_rows.append(row)
# consistent_scores = [1.0 if row['consistent_hit'] >= 1.0 else 0.0 for row in new_rows]
t_batch = pa.Table.from_pylist(new_rows)
# t_batch = t_batch.append_column("consistent_score",pa.array(consistent_scores))
print(t_batch)


# Usage:
# df_reviews_metrics = add_consistent_score(df_reviews_metrics)

pyarrow.Table
item_id: string
ori_semi_positive_review_list: list<item: string>
  child 0, item: string
ori_positive_review_distinct: list<item: string>
  child 0, item: string
positive_points: list<item: string>
  child 0, item: string
negative_points: list<item: string>
  child 0, item: string
hard_negative_query: list<item: string>
  child 0, item: string
constraint_list: list<item: string>
  child 0, item: string
consistent_hit: int64
consistent_score: double
----
item_id: [["B0B186FXGM","B099KSC4L2","B08MF3W9DZ","B0B34GRFL2","B0B86TKMP8","B0BXXD1FZ4","B08L5KN7X4","B00K7J4S6G","B09TT8ZVPN"]]
ori_semi_positive_review_list: [[["The masks are really good for hydrating skin and moisturizing. They are super easy to use and fit really well. My mother uses this every morning and she highly recommends it.","While the mask was a tiny bit small compared to most brands I have tried, it had plenty of serum and left my skin looking and feeling very hydrated and soothed. I knew it has tea tree o

In [74]:
df_reviews_metrics.group_by("item_id").aggregate([("hard_negative_query", "count")])

pyarrow.Table
item_id: string
hard_negative_query_count: int64
----
item_id: [["B0B186FXGM","B099KSC4L2","B08MF3W9DZ","B0B34GRFL2","B0B86TKMP8","B0BXXD1FZ4","B08L5KN7X4","B00K7J4S6G","B09TT8ZVPN"]]
hard_negative_query_count: [[1,1,1,1,1,1,1,1,1]]

In [69]:
t_batch.group_by("item_id").aggregate([("consistent_score", "mean")])

pyarrow.Table
item_id: string
consistent_score_mean: double
----
item_id: [["B0B186FXGM","B099KSC4L2","B08MF3W9DZ","B0B34GRFL2","B0B86TKMP8","B0BXXD1FZ4","B08L5KN7X4","B00K7J4S6G","B09TT8ZVPN"]]
consistent_score_mean: [[0.4633733332157135,0,0.2809489071369171,0.5029644966125488,0.2630251348018646,0.7017921209335327,0.6043304800987244,0.3662206530570984,0.32680821418762207]]

3 - Check the user intent score

In [46]:
import outlines, pathlib

try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"/content/prompts/system_prompt_generate_user_intent_list.txt"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    system_template = \
    """
    ```json
         {
        "task": "You are a user intent extractor. You recieve a text from a user that describes a product they review or a search query. Your task is to identify **what** the user wants from the product. — i.e., what the user fundamentally expects, requires, or rejects in a product.",
        "instructions": [
        "1. Identify **only explicit or strongly implied intents** — do NOT infer preferences beyond the text.",
        "2. For each expectation or requirement identified, create an intent object with the following structure:",
        "   - `intent`: a concise, canonical phrase describing the user's expectation (noun-phrase preferred, e.g., 'good grip on wet surfaces').",
        "3. Prioritize *avoidance* and *constraint* intents — they are critical for query rewriting and hard negative generation.",
        "4. The input type is describe by the text_type parameter. The input can be a review or a search query",
        "5. If the input is a review, distinguish between:",
        "   - Desired traits (what worked or was expected)",
        "   - Undesired traits (what failed or must be avoided in next purchase)."
        ],

        "input_format": {
          "text": "string — the user's natural-language product search query",
          "text_type": "string - review or search query"
        },
        "output_type": "json",
        "output_format": {
          "intents": [
            "string — a text that summarizes what the user expects from the product. ",
            "Avoid vague terms; prefer concrete, actionable constraints derived directly from the query."
          ]
        },
        "requirements": [
        "Return only the JSON format as specified; do not include any additional text or explanations.",
        "Do not include vague intents like 'good quality' or 'reliable' unless explicitly defined in the text.",
        "Rephrase avoidance intents as *positive requirements* where possible (e.g., 'no slipping' → 'non-slip grip').",
        "Use **English** for `intent` field (to improve cross-lingual embedding consistency), even if input is in Portuguese.",
        "Maximum 6 intents. Prefer specificity over exhaustiveness."
    ]
      }

    ```
    """

    with open('/content/prompts/system_prompt_generate_user_intent_list.txt', 'w+') as f:
          f.write(system_template)
          f.close()
          upload_file(
              path_or_fileobj=f"/content/prompts/system_prompt_generate_user_intent_list.txt",
              path_in_repo=f"prompts/system_prompt_generate_user_intent_list.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add system prompt for user intent score rate"
          )

In [42]:
import outlines, pathlib

try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"/content/prompts/user_prompt_generate_user_intent_list.txt"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    user_template = \
    """

    ```json
    {
      text: {{text}},
      text_type: {{text_type}}
    }
    ```

    """

    with open('/content/prompts/user_prompt_generate_user_intent_list.txt', 'w+') as f:
          f.write(user_template)
          f.close()
          upload_file(
              path_or_fileobj=f"/content/prompts/user_prompt_generate_user_intent_list.txt",
              path_in_repo=f"prompts/user_prompt_generate_user_intent_list.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add user prompt for user intent score rate"
          )

In [52]:
# generate user intent from query
generate_intent_list(
    df_reviews_w_metrics,
    model,
    tokenizer,
    pydantic_model=Intents,
    system_template_path='/content/prompts/system_prompt_generate_user_intent_list.txt',
    user_template_path='/content/prompts/user_prompt_generate_user_intent_list.txt',
    text_type="review",
)


```json
{
  "intents": [
    "needs stronger bristles to remove tangles",
    "requires a non-gentle brushing experience",
    "wants bristles that are tougher"
  ]
}
```
intents=['needs stronger bristles to remove tangles', 'requires a non-gentle brushing experience', 'wants bristles that are tougher']
```json
{
  "intents": [
    "non-pokey bristles",
    "helpful in removing tangled hair",
    "use of a separate comb"
  ]
}
```
intents=['non-pokey bristles', 'helpful in removing tangled hair', 'use of a separate comb']
Batch 0 done.


In [55]:
# generate user review intent from query
generate_intent_list(
    df_reviews_w_metrics,
    model,
    tokenizer,
    pydantic_model=Intents,
    system_template_path='/content/prompts/system_prompt_generate_user_intent_list.txt',
    user_template_path='/content/prompts/user_prompt_generate_user_intent_list.txt',
    text_type="search_query",
)

```json
{
  "intents": [
    "lightweight and flexible design",
    "tough enough bristles to handle tangles",
    "not soft bristles",
    "avoiding brushes with soft bristles"
  ]
}
```
intents=['lightweight and flexible design', 'tough enough bristles to handle tangles', 'not soft bristles', 'avoiding brushes with soft bristles']
```json
{
  "intents": [
    "lightweight and flexible hairbrush",
    "hairbrush with strong bristles",
    "hairbrush suitable for curly hair",
    "non-flexible hairbrush",
    "hairbrush with soft bristles"
  ]
}
```
intents=['lightweight and flexible hairbrush', 'hairbrush with strong bristles', 'hairbrush suitable for curly hair', 'non-flexible hairbrush', 'hairbrush with soft bristles']
Batch 0 done.


In [73]:
def calculate_intent_consistent_score(df_reviews):
  new_rows = []
  for batch in df_reviews.to_batches():
    for row in batch.to_pylist():
      user_review_intent = row['user_review_intent']
      user_query_intent = row['user_query_intent']
      row["user_intent_score"] = compute_consistent_score(user_query_intent,user_review_intent)
      new_rows.append(row)
  new_batch = pa.Table.from_pylist(new_rows)
  return new_batch

df_raw_reviews_intent = pq.read_table("/content/datasets/user_intent/review/all_Beauty_part_0.parquet")
df_raw_user_intent = pq.read_table("/content/datasets/user_intent/search_query/all_Beauty_part_0.parquet")

df_raw_reviews_intent = df_raw_reviews_intent.sort_by("item_id")
df_raw_user_intent = df_raw_user_intent.sort_by("item_id")

df_reviews_w_metrics = df_raw_reviews_intent.append_column("user_query_intent",df_raw_user_intent.column("user_query_intent"))
df_reviews_w_metrics = calculate_intent_consistent_score(df_reviews_w_metrics)
print(df_reviews_w_metrics)

pyarrow.Table
item_id: string
ori_positive_review: string
ori_semi_positive_review: string
positive_points: list<item: string>
  child 0, item: string
negative_points: list<item: string>
  child 0, item: string
hard_negative_query: string
constraints: list<item: string>
  child 0, item: string
consistent_score: double
consistent_hit: double
user_review_intent: list<item: string>
  child 0, item: string
user_query_intent: list<item: string>
  child 0, item: string
user_intent_score: double
----
item_id: [["B08MF3W9DZ","B08MF3W9DZ"]]
ori_positive_review: [["Great brushes  for curly hair Love these brushes for my girls who have super curly hair that’s very hard to untangle. Works great on wet or  dry Hair","Great brushes  for curly hair Love these brushes for my girls who have super curly hair that’s very hard to untangle. Works great on wet or  dry Hair"]]
ori_semi_positive_review: [["It’s a great brush, but it needs to be a tad bit tougher…. It combs, but when you need to get a tangle o

In [78]:
print(df_reviews_w_metrics.column("user_intent_score"))

[
  [
    0.8693621158599854,
    0.6950052976608276
  ]
]


In [74]:
df_reviews_w_metrics.group_by("item_id").aggregate([("user_intent_score", "mean")])

pyarrow.Table
item_id: string
user_intent_score_mean: double
----
item_id: [["B08MF3W9DZ"]]
user_intent_score_mean: [[0.7821837067604065]]

In [61]:
df_raw_reviews_intent = pq.read_table("/content/datasets/user_intent/review/all_Beauty_part_0.parquet")
df_raw_reviews_intent

pyarrow.Table
item_id: string
ori_positive_review: string
ori_semi_positive_review: string
positive_points: list<element: string>
  child 0, element: string
negative_points: list<element: string>
  child 0, element: string
hard_negative_query: string
constraints: list<element: string>
  child 0, element: string
consistent_score: double
consistent_hit: double
user_review_intent: list<element: string>
  child 0, element: string
----
item_id: [["B08MF3W9DZ","B08MF3W9DZ"]]
ori_positive_review: [["Great brushes  for curly hair Love these brushes for my girls who have super curly hair that’s very hard to untangle. Works great on wet or  dry Hair","Great brushes  for curly hair Love these brushes for my girls who have super curly hair that’s very hard to untangle. Works great on wet or  dry Hair"]]
ori_semi_positive_review: [["It’s a great brush, but it needs to be a tad bit tougher…. It combs, but when you need to get a tangle out, the bristles aren’t tough enough. I have to literally go get

In [81]:
# # Force garbage collection to free CPU memory
# import gc
# gc.collect()

# # Clear GPU cache to free up CUDA memory
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()
#     print("✅ GPU cache cleared")
# else:
#     print("CUDA not available")

# print("✅ Cleanup complete - model/tokenizer removed from CPU/GPU")

✅ GPU cache cleared
✅ Cleanup complete - model/tokenizer removed from CPU/GPU
